In [ ]:
import os
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import PolynomialFeatures

# Regression

In [ ]:
def generateNoisyData(num_points=50):
    """
    Generates noisy sample points and saves the data. The function will save the data as a npz file.
    Args:
        num_points: number of sample points to generate.
    """
    x = np.linspace(1, 4*math.pi, num_points)
    y = np.sin(x*0.5)

    # Define the noise model
    nmu = 0
    sigma = 0.3
    noise = nmu + sigma * np.random.randn(num_points)
    t = y + noise

    # Save the data
    np.savez('data.npz', x=x, y=y, t=t, sigma=sigma)

# Feel free to change aspects of this function to suit your needs.
# Such as the title, labels, colors, etc.
def plot_with_shadded_bar(x=None, y=None, sigma=None):
    """
    Plots the GT data for visualization.
    Args:
        x: x values
        y: y values
        sigma: standard deviation
    """
    if not os.path.exists('results'):
        os.makedirs('results')

    # Example plotting with the GT data, you can use this as a reference. You will later 
    # use this function to plot the results of your model.
    np.load('data.npz')
    x = np.load('data.npz')['x']
    y = np.load('data.npz')['y']
    t = np.load('data.npz')['t']
    sigma = np.load('data.npz')['sigma']

    # Plot the ground truth curve of the generated data.
    fig, ax = plt.subplots()

    # Plot curve with red shaded region spans on std.
    ax.plot(x, y, 'r', label='Ground Truth Example')
    ax.fill_between(x, y-sigma, y+sigma, color='r', alpha=0.2)

    # Plot the noisy data points.
    ax.scatter(x, t, label='Noisy Data')

    ax.set_xlabel('x')
    ax.set_ylabel('t')
    ax.set_title('Data Point Plot Example')

    plt.savefig('results/gt_data.png')
    plt.close(fig)

def plot_results(x=None, y=None, pred1 = None, label1 = None, pred2 = None, label2 = None, title = None, file_name = None):
    """
    Plots the GT data for visualization.
    Args:
        x: x values
        y: y values
        pred: prediction values
    """
    if not os.path.exists('results'):
        os.makedirs('results')

    np.load('data.npz')
    x = np.load('data.npz')['x']
    y = np.load('data.npz')['y']
    t = np.load('data.npz')['t']

    # Plot the ground truth curve of the generated data.
    fig, ax = plt.subplots()

    ax.plot(x, y, 'g', label = "Ground Truth")
    ax.plot(x, pred1, 'r', label = label1)
    if pred2 is not None:
        ax.plot(x, pred2, "y", label = label2)

    # Plot the noisy data points.
    ax.scatter(x, t, label='Noisy Data')

    ax.set_xlabel('X Values')
    ax.set_ylabel('Y Values, Predictions, Noisy Data')
    ax.set_title(title)
    plt.legend()
    plt.savefig(f'results/{file_name}.png')
    plt.show()
    plt.close(fig)

In [ ]:
class ML():
    def __init__(self, degree = None):
        """
        Initializes the Maximum Likelihood model. You may add any parameters/variables you need.
        You may also add any functions you may need to the class.
        """
        self.degree = degree

    def fit(self, x, y):
        """
        Fits the Maximum Likelihood model to the data.
        Args:
            x (np.array): The features of the data.
            y (np.array): The targets of the data.
        """
        x = PolynomialFeatures(degree = self.degree).fit_transform(x.reshape(-1,1))
        intermediate = (x.T)@x
        weights = np.linalg.solve(intermediate, (x.T)@y)
        return weights

    def predict(self, x, weights):
        """
        Predicts the targets of the data.
        Args:
            x (np.array)      : The features of the data.
            weights (np.array): Weights from the ML.fit() method.
        Returns:
            np.array: The predicted targets of the data.
        """
        x = PolynomialFeatures(degree = self.degree).fit_transform(x.reshape(-1,1))
        predictions = np.dot(x, weights)
        return predictions

In [ ]:
class MAP():
    def __init__(self, alpha=0.005, beta=11.1, lnlambda = None, customReguralization = False, degree = None):
        """
        Initializes the Maximum A Posteriori model. You may add any parameters/variables you need.
        Args:
            alpha (float): The alpha parameter of the model.
            beta (float): The beta parameter of the model.

        You may also add any functions you may need to the class.
        """
        self.alpha = alpha
        self.beta = beta
        self.lnlambda = lnlambda
        self.customReguralization = customReguralization
        self.degree = degree

    def fit(self, x, y):
        """
        Fits the Maximum A Posteriori model to the data.
        Args:
            x (np.array): The features of the data.
            y (np.array): The targets of the data.
        """
        if self.customReguralization:
            lnlambda = self.lnlambda
        else:
            lnlambda = np.log(self.alpha/self.beta)

        x = PolynomialFeatures(degree = self.degree).fit_transform(x.reshape(-1,1))
        intermediate = ((x.T)@x) + np.exp(lnlambda)*np.eye(x.shape[1], x.shape[1])
        weights = np.linalg.solve(intermediate, (x.T)@y)
        return weights

    def predict(self, x, weights):
        """
        Predicts the targets of the data.
        Args:
            x (np.array): The features of the data.
            weights (np.array): The weights from the MAP.fit() method.
        Returns:
            np.array: The predicted targets of the data.
        """
        x = PolynomialFeatures(degree = self.degree).fit_transform(x.reshape(-1,1))
        predictions = np.dot(x, weights)
        return predictions

In [ ]:
generateNoisyData(50)
plot_with_shadded_bar()

In [ ]:
np.load('data.npz')
x = np.load('data.npz')['x']
y = np.load('data.npz')['y']
t = np.load('data.npz')['t']
sigma = np.load('data.npz')['sigma']

In [ ]:
ML_Model = ML(degree = 3)
ml_weights = ML_Model.fit(x, t)
ml_predictions = ML_Model.predict(x, ml_weights)
plot_results(x, y, ml_predictions, title = "ML Model Degree 3", file_name = "ml_3", label1 = "ML Model Predictions")

MAP_Model = MAP(degree = 3, customReguralization = False)
map_weights = MAP_Model.fit(x, t)
map_predictions = MAP_Model.predict(x, map_weights)
plot_results(x, y, map_predictions, title = "MAP Model Degree 3", file_name = "map_3", label1 = "MAP Model Predictions")

plot_results(x, y, pred1 = ml_predictions, label1 = "ML Model Degree 3", pred2 = map_predictions, label2 = "MAP Model Degree 3", title = "ML vs MAP Predictions", file_name = "mlevsmap")

lnlambda = -18
CustomModel = MAP(degree = 3, customReguralization = True, lnlambda = lnlambda)
custom_weights = CustomModel.fit(x, t)
custom_predictions = CustomModel.predict(x, custom_weights)
plot_results(x, y, custom_predictions, title = r"Custom Model Degree 3, ln$\lambda$ = " + str(lnlambda), file_name = "lnlambda-18", label1 = r"$ln\lambda$ = "+str(lnlambda)+" Model")

# Classification

In [ ]:
def viz_desc_bounds(classifier, feats, labels, idxA, idxB):
    """
    Visualizes the decision boundaries of a classifier trained on two features of the dataset.
    Args:
        classifier: linear classifier trained on 2 features.
        feats: features to be used for visualization.
        labels: labels to be used for visualization.
        idxA & idxB: indices of the features to be used for visualization. 
    """
    if not os.path.exists('results'):
        os.makedirs('results')

    ys = np.sort(np.unique(labels))
    y_ind = np.searchsorted(ys, labels)

    fig, ax = plt.subplots()

    x0, x1 = feats[:, 0], feats[:, 1]
    all_feats = np.concatenate((x0, x1))
    pad = np.percentile(all_feats, 60)

    x_min, x_max = x0.min() - pad, x0.max() + pad
    y_min, y_max = x1.min() - pad, x1.max() + pad
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01), np.arange(y_min, y_max, 0.01))

    preds = classifier.predict(np.c_[xx.ravel(), yy.ravel()])
    preds = preds.reshape(xx.shape)

    lut = np.sort(np.unique(labels)) 
    ind = np.searchsorted(lut,preds)

    markers = ["o", "v", "P", "X", "s", "p", "h", "d"]
    ax.contourf(xx, yy, preds, cmap=plt.cm.Pastel1, alpha=0.8)
    for i in range(len(lut)):
        ax.scatter(x0[y_ind == i], x1[y_ind == i], color=plt.cm.jet(i/len(lut)), s=50, edgecolors='k', marker=markers[i])

    ax.set_xlabel(f'Feature {idxA}')
    ax.set_ylabel(f'Feature {idxB}')
    ax.set_title('Decision Boundary')

    handles = []
    markers = ["o", "v", "P", "X", "s", "p", "h", "d"]
    handles = [plt.plot([],[],color=plt.cm.jet(i/len(lut)), ls="", marker=markers[i])[0] for i in range(len(lut))]
    labels = [f'Class {i}' for i in lut]
    ax.legend(handles, labels, loc='upper right')
    plt.show()
    # plt.savefig('results/decision_boundary.png')

In [ ]:
def load_dataset(dataset='taiji', verbose=False, subject_index=3):
    '''
    Loads the taiji dataset.
    Args:
        dataset: name of the dataset to load. Currently only taiji is supported.
        verbose: print dataset information if True.
        subject_index: subject index to use for LOSO. The subject with this index will be used for testing. 

    Returns (all numpy arrays):
        train_feats: training features.
        train_labels: training labels.
        test_feats: testing features.
        test_labels: testing labels.
    '''

    if dataset == 'taiji':
        labels = np.loadtxt("../P1/data/taiji/taiji_labels.csv", delimiter=",", dtype=int)
        person_idxs = np.loadtxt("../P1/data/taiji/taiji_person_idx.csv", delimiter=",", dtype=int)
        feats = np.loadtxt("../P1/data/taiji/taiji_quat.csv", delimiter=",", dtype=float)

        # Combine repeated positions
        labels[labels == 4] = 2
        labels[labels == 8] = 6

        # Remove static dimensions. Get mask of all features with zero variance
        feature_mask = np.var(feats, axis=1) > 0

        # Train mask
        train_mask = person_idxs != subject_index

        train_feats = feats[feature_mask, :][:, train_mask].T
        train_labels = labels[train_mask].astype(int)
        test_feats = feats[feature_mask, :][:, ~train_mask].T
        test_labels = labels[~train_mask].astype(int)


    if verbose:
        print(f'{dataset} Dataset Loaded')
        print(f'\t# of Classes: {len(np.unique(train_labels))}')
        print(f'\t# of Features: {train_feats.shape[1]}')
        print(f'\t# of Training Samples: {train_feats.shape[0]}')
        print('\t# per Class in Train Dataset:')
        for cls in np.unique(train_labels):
            print (f'\t\tClass {cls}: {np.sum(train_labels == cls)}')
        print(f'\t# of Testing Samples: {test_feats.shape[0]}')
        print('\t# per Class in Test Dataset:')
        for clas in np.unique(test_labels):
            print(f'\t\tClass {clas}: {np.sum(test_labels == clas)}')
        
    return train_feats, train_labels, test_feats, test_labels

def plot_conf_mats(dataset, **kwargs):
    """
    Plots the confusion matrices for the training and testing data.
    Args:
        dataset: name of the dataset.
        train_labels: training labels.
        pred_train_labels: predicted training labels.
        test_labels: testing labels.
        pred_test_labels: predicted testing labels.
    """

    train_labels = kwargs['train_labels']
    pred_train_labels = kwargs['pred_train_labels']
    test_labels = kwargs['test_labels']
    pred_test_labels = kwargs['pred_test_labels']

    train_confusion = confusion_matrix(train_labels, pred_train_labels)
    test_confusion = confusion_matrix(test_labels, pred_test_labels)

    # Plot the confusion matrices as seperate figures
    fig, ax = plt.subplots()
    disp = ConfusionMatrixDisplay(confusion_matrix=train_confusion, display_labels=np.unique(train_labels))
    disp.plot(ax=ax, xticks_rotation='vertical')
    ax.set_title('Training Confusion Matrix')
    plt.tight_layout()
    plt.savefig(f'results/{dataset}_train_confusion.png', bbox_inches='tight', pad_inches=0)

    fig, ax = plt.subplots()
    disp = ConfusionMatrixDisplay(confusion_matrix=test_confusion, display_labels=np.unique(test_labels))
    disp.plot(ax=ax, xticks_rotation='vertical')
    ax.set_title('Testing Confusion Matrix')
    plt.tight_layout()
    plt.show()
    # plt.savefig(f'results/{dataset}_test_confusion.png', bbox_inches='tight', pad_inches=0)

def example_decision_boundary(dataset='taiji', indices=[0, 6]):
    """
    An example of how to visualize the decision boundary of a classifier.
    """
    train_feats, train_labels, test_feats, test_labels = load_dataset(dataset=dataset)

    dc_train_feats = train_feats[:, indices]
    dc_test_feats = test_feats[:, indices]

    # Example Linear Discriminant Analysis classifier with sklearn's implemenetation
    clf = LinearDiscriminantAnalysis()
    clf.fit(dc_train_feats, train_labels)

    # Visualize the decision boundary
    viz_desc_bounds(clf, dc_test_feats, test_labels, indices[0], indices[1])

def example_classification(dataset='taiji'):
    """
    An example of performing classification. Except you will need to first project the data.
    """
    train_feats, train_labels, test_feats, test_labels = load_dataset(dataset=dataset)

    # Example Linear Discriminant Analysis classifier with sklearn's implemenetation
    clf = LinearDiscriminantAnalysis()
    clf.fit(train_feats, train_labels)

    # Predict the labels of the training and testing data
    pred_train_labels = clf.predict(train_feats)
    pred_test_labels = clf.predict(test_feats)

    # Get statistics
    plot_conf_mats(dataset, train_labels=train_labels, pred_train_labels=pred_train_labels, test_labels=test_labels, pred_test_labels=pred_test_labels)

In [ ]:
example_decision_boundary()

In [ ]:
example_classification()